In [1]:
from main import agent_executor

/opt/miniconda3/envs/internal-pm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/miniconda3/envs/internal-pm/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/miniconda3/envs/internal-pm/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)


In [2]:
for chunk in agent_executor.stream({"input": "What are the latest tickets from Zendesk?"}):
    # Print agent actions (tool calls)
            if "actions" in chunk:
                for action in chunk["actions"]:
                    print(f"\n🔧 Tool Call: {action.tool}")
                    print(f"📥 Input: {action.tool_input}")
            
            # Print tool observations (tool outputs)
            if "steps" in chunk:
                for step in chunk["steps"]:
                    print(f"\n📤 Tool Output ({step.action.tool}):")
                    print(f"{step.observation}")
            
            # Print final output
            if "output" in chunk:
                print(f"\n✅ Final Answer:")
                print(f"{chunk['output']}")
                print("\n" + "="*60 + "\n")



> Entering new None chain...

🔧 Tool Call: fetch_schema_details
📥 Input: ZendeskTickets
Thought: The user is asking for the latest tickets from Zendesk. I should first check the schema details for Zendesk to understand the available tables, and then query the data to find the latest tickets.

Action: fetch_schema_details
Action Input: ZendeskTickets

/opt/miniconda3/envs/internal-pm/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my-incorta.cloud.incorta.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'schema_details': {'name': 'ZendeskTickets', 'schemaId': 273, 'ownerId': 883, 'ownerName': 'My Incorta', 'permissionCode': '0', 'tables': [{'name': 'ticket_comments', 'type': 'view', 'alias': False, 'multiDataSource': False, 'cached': True, 'snapshot': False, 'enablePkIndexOnFullLoad': True, 'directoryBased': False, 'maxColumnNumber': 8, 'connectorId': -1, 'loadOrder': 1, 'partialLoading': 'none', 'dataRetention': 'none', 'deletionDataSetEnabled': False, 'cascadePurge': False, 'columns': [{'name': 'id', 'label': 'Id', 'function': 'key', 'hide': False, 'encrypt': False, 'autoDetectType': True, 'isPrimaryKey': False, 'isForeignKey': False, 'index': 1, 'physicalName': 'id', 'length': 0, 'dateFormat': '', 'mappedDataType': 'long'}, {'name': 'ticket_id', 'label': 'Ticket Id', 'function': 'dimension', 'hide': False, 'encrypt': False, 'autoDetectType': True, 'isPrimaryKey': False, 'isForeignKey': False, 'index': 2, 'physicalName': 'ticket_id', 'length': 0, 'dateFormat': '', 'mappedDataType':

/opt/miniconda3/envs/internal-pm/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my-incorta.cloud.incorta.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'data': {'metadata': [{'label': 'id', 'dataType': 'bigint'}, {'label': 'subject', 'dataType': 'string'}, {'label': 'status', 'dataType': 'string'}, {'label': 'created_at', 'dataType': 'timestamp'}, {'label': 'updated_at', 'dataType': 'timestamp'}], 'data': [[{'value': '60969'}, {'value': '7P Analytics is not starting'}, {'value': 'open'}, {'value': '2025-10-29 13:25:36.0'}, {'value': '2025-10-29 13:35:11.0'}], [{'value': '60968'}, {'value': 'Facing issue while loading data from OERP in Oprod Pstage'}, {'value': 'pending'}, {'value': '2025-10-29 12:56:38.0'}, {'value': '2025-10-29 13:12:50.0'}], [{'value': '60967'}, {'value': 'Postgres failure'}, {'value': 'pending'}, {'value': '2025-10-29 12:01:56.0'}, {'value': '2025-10-29 12:51:17.0'}], [{'value': '60966'}, {'value': '[CloudOps Spin-off] 60962 - [Baja Food Industries Co.]: Notebook is not working'}, {'value': 'pending'}, {'value': '2025-10-29 11:55:25.0'}, {'value': '2025-10-29 13:08:47.0'}], [{'value': '60965'}, {'value': 'oracle t

In [3]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

env_url = os.getenv("INCORTA_ENV_URL")
tenant = os.getenv("INCORTA_TENANT")
user = os.getenv("INCORTA_USERNAME")
password = os.getenv("INCORTA_PASSWORD")

def login_with_credentials(env_url, tenant, user, password):
    """
    Logs into Incorta using username, password, and tenant credentials.
    Returns a session with relevant authentication details.
    """
    response = requests.post(
        f"{env_url}/authservice/login",
        data={"tenant": tenant, "user": user, "pass": password},
        verify=True,
        timeout=60
    )

    if response.status_code != 200:
        response.raise_for_status()

        # Extract session cookies
    id_cookie, login_id = None, None
    for item in response.cookies.items():
        if item[0].startswith("JSESSIONID"):
            id_cookie, login_id = item
            break

    if not id_cookie or not login_id:
        raise Exception("Failed to retrieve session cookies during login.")

        # Verify login and retrieve CSRF token
    response = requests.get(
        f"{env_url}/service/user/isLoggedIn",
        cookies={id_cookie: login_id},
        verify=True,
        timeout=60
    )

    if response.status_code != 200 or "XSRF-TOKEN" not in response.cookies:
        raise Exception(f"Failed to log in to {env_url} for tenant {tenant} using user {user}. Please verify credentials.")

        # Retrieve CSRF token and access token
    csrf_token = response.cookies["XSRF-TOKEN"]
    authorization = response.json().get("accessToken")

    if not authorization:
        raise Exception("Failed to retrieve access token during login.")

    return {
        "env_url": env_url,
        "id_cookie": id_cookie,
        "id": login_id,
        "csrf": csrf_token,
        "authorization": authorization,
        "verify": True,
        "session_cookie": {id_cookie: login_id, "XSRF-TOKEN": csrf_token}
    }


def fetch_schema_details(schema_name: str):
    """Fetch schema details from the Incorta environment."""

    login_creds = login_with_credentials(env_url, tenant, user, password)

    url = f"{env_url}/bff/v1/schemas/name/{schema_name}"

    cookie = ""
    for key, value in login_creds['session_cookie'].items():
        cookie += f"{key}={value};"

    headers = {
        "Authorization": f"Bearer {login_creds['authorization']}",
        "Content-Type": "application/json",
        "X-XSRF-TOKEN": login_creds["csrf"],
        "Cookie": cookie
    }

    response = requests.get(url, headers=headers, verify=False)

    if response.status_code == 200:
        response_data = response.json()

        return {"schema_details": response_data}
    else:
        return {"error": f"Failed to fetch schema details: {response.status_code} - {response.text}"}

In [5]:
fetch_schema_details("ZendeskTickets")

/opt/miniconda3/envs/internal-pm/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my-incorta.cloud.incorta.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'schema_details': {'name': 'ZendeskTickets',
  'schemaId': 273,
  'ownerId': 883,
  'ownerName': 'My Incorta',
  'permissionCode': '0',
  'tables': [{'name': 'ticket_comments',
    'type': 'view',
    'alias': False,
    'multiDataSource': False,
    'cached': True,
    'snapshot': False,
    'enablePkIndexOnFullLoad': True,
    'directoryBased': False,
    'maxColumnNumber': 8,
    'connectorId': -1,
    'loadOrder': 1,
    'partialLoading': 'none',
    'dataRetention': 'none',
    'deletionDataSetEnabled': False,
    'cascadePurge': False,
    'columns': [{'name': 'id',
      'label': 'Id',
      'function': 'key',
      'hide': False,
      'encrypt': False,
      'autoDetectType': True,
      'isPrimaryKey': False,
      'isForeignKey': False,
      'index': 1,
      'physicalName': 'id',
      'length': 0,
      'dateFormat': '',
      'mappedDataType': 'long'},
     {'name': 'ticket_id',
      'label': 'Ticket Id',
      'function': 'dimension',
      'hide': False,
      'encry